# Counterfactuals guided by prototypes on Boston housing dataset

This notebook goes through an example of [prototypical counterfactuals](../methods/CFProto.ipynb) using [k-d trees](https://en.wikipedia.org/wiki/K-d_tree) to build the prototypes. Please check out [this notebook](./cfproto_mnist.ipynb) for a more in-depth application of the method on MNIST using (auto-)encoders and trust scores.

In this example, we will train a simple neural net to predict whether house prices in the Boston area are above the median value or not. We can then find a counterfactual to see which variables need to be changed to increase or decrease a house price above or below the median value.

In [1]:
import tensorflow as tf
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs 
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from sklearn.datasets import load_boston
from alibi.explainers.cfproto import CounterfactualProto
from sklearn.preprocessing import StandardScaler, MinMaxScaler

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False

TF version:  2.5.1
Eager execution enabled:  False


## Load and prepare Boston housing dataset

In [2]:
boston = load_boston()
data = boston.data
target = boston.target
feature_names = boston.feature_names

Transform into classification task: target becomes whether house price is above the overall median or not

In [3]:
y = np.zeros((target.shape[0],))
y[np.where(target > np.median(target))[0]] = 1

Remove categorical feature

In [4]:
data = np.delete(data, 3, 1)
feature_names = np.delete(feature_names, 3)

Explanation of remaining features:

- CRIM: per capita crime rate by town
- ZN: proportion of residential land zoned for lots over 25,000 sq.ft.
- INDUS: proportion of non-retail business acres per town
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centres
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per USD10,000
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population

Standardize data

In [5]:
random.seed(0)
a = list(range(len(data)))
random.shuffle(a)

length = len(a)

Define train and test set

In [6]:
train_x, train_y = data[a[0:int(0.5*length)]], y[a[0:int(0.5*length)]]
query_x, query_y = data[a[int(0.5*length):int(0.75*length)]], y[a[int(0.5*length):int(0.75*length)]]
test_x, test_y = data[a[int(0.75*length):]], y[a[int(0.75*length):]]

In [7]:
scaler = StandardScaler()
strain_x = scaler.fit_transform(train_x)
squery_x = scaler.transform(query_x)
stest_x = scaler.transform(test_x)

In [8]:
otrain_y = to_categorical(train_y)
oquery_y = to_categorical(query_y)
otest_y = to_categorical(test_y)

In [9]:
nn = load_model('nn_boston.h5')
print(nn.evaluate(squery_x, oquery_y))
print(nn.evaluate(stest_x, otest_y))

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


[0.23143865854021103, 0.8730159]
[0.31804604793158103, 0.8582677]


In [10]:
query_p = np.argmax(nn.predict(squery_x), axis = 1)
test_p = np.argmax(nn.predict(stest_x), axis = 1)

In [11]:
print(test_p)

[1 0 0 0 0 1 1 1 0 0 1 1 1 1 0 1 0 1 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 1 1 1 1
 0 1 0 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 1 0 0 0 1 0 1 0 0 1 0 0 0 1 0 1 0 1 0
 0 0 1 1 1 1 0 1 1 1 0 0 1 0 1 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1
 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 0]


In [12]:
query_tmp = np.concatenate((query_x, query_p[:, np.newaxis]), axis = 1)
test_tmp = np.concatenate((test_x, test_p[:, np.newaxis]), axis = 1)

In [13]:
np.save("boston_housing_query.npy", query_tmp)
np.save("boston_housing_test.npy", test_tmp)